[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# Control de acceso mediante usuario y contraseña.

## La extensión *Flask-Security*.

Flask cuenta con varias extensiones enfocadas a garantizar la seguridad de los servicios web. Una de ellas es [*Flask-Security*](https://pythonhosted.org/Flask-Security/), la cual a su vez hace uso de las extensiones y módulos:

* [Flask-Login](https://flask-login.readthedocs.io/).
* [Flask-Mail](https://pythonhosted.org/Flask-Mail/).
* [Flask-Principal](https://pythonhosted.org/Flask-Principal/).
* [Flask-WTF](https://flask-wtf.readthedocs.io/).
* [itsdangerous](https://itsdangerous.palletsprojects.com).
* [passlib](https://passlib.readthedocs.io/).

Es necesario contar con un sistema de generación de *hash* como [*bcrypt*](https://pypi.org/project/bcrypt).

Esta extensión controla el proceso de alta, registro e incluso recuperación por correo electrónico de usuarios, contraseñas y roles.

In [ ]:
!pip install flask-security flask-sqlalchemy bcrypt email-validator

## Ejemplo de una aplicación básica con *Flask-Security* y el *Flask-SQLAlchemy*.

Esta es una versión modificada del ejemplo localizado en https://pythonhosted.org/Flask-Security/quickstart.html#sqlalchemy-application. 

El uso de *Flask-Security* es aún más extenso, pero para los fines de este curso sólo se explorarán los aspectos más básicos.

### Importación de los módulos y paquetes.

* La extensión cuenta con objetos capaces de conectarse al ORM de *SQLAlchemy* y crear tablas especializadas para gestión de usuarios y roles en la base de datos.
* El uso de *mixins* facilita la creación de clases que contengan métodos compatibles con la extensión.

In [ ]:
from flask import Flask, session, 
from flask_sqlalchemy import SQLAlchemy
from flask_security import Security, SQLAlchemyUserDatastore, \
    UserMixin, RoleMixin, login_required

In [ ]:
!rm data/alumnos.sqlite

### Creación y configuración de la aplicación de Flask.

Para poder utilizar *Flask-Security* es necesario incluir algunos parámetros de configuración de *Flask-SQLAlchemy* tales como:

* ```'SECRET_KEY'```, el cual es utilizado por *Flask-WTF* para saegurar los formularios.
* ```'SECURITY_PASSWORD_SALT'```, el cual sirve para cifrar las contraseñas.

In [ ]:
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secretísimo'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///data/alumnos.sqlite'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECURITY_PASSWORD_SALT'] = 'ultra-secreto'
db = SQLAlchemy(app)

## Definición de las tablas de usuarios y roles.

Se crearán varias tablas y clases.


**Advertencia:** para evitar conflictos es necesario eliminar el archivo ```usuarios.db``` localizado en el directorio [```data```](data) en caso de que exista.

### Creación de la tabla ```roles``` con la clase ```db.Table```. 

La clase ```db.Table()``` permite definir un objeto ligado a una tabla en la base de datos, ingresando objetos instanciados de ```db.Column``` como argumentos para el método ```__init__()``` del objeto instanciado.

Cabe hacer notar que la tabla ```roles``` contiene las columnas ```id_user``` e ```id_role```, las cuales están ligadas a los atributos ```user.id``` y ```role.id``` respectivamente.

La clase ```db.ForeginKey``` liga a un campo ligado a la columna de otra tabla.

In [ ]:
class RolesUsers(db.Model) :
    __tablename__ = 'roles_users'
    id = db.Column(db.Integer(), primary_key=True)
    user_id = db.Column('user_id', db.Integer(), db.ForeignKey('user.id'))
    role_id = db.Column('role_id', db.Integer(), db.ForeignKey('role.id'))

### Creación de las tablas ```role``` y ```user``` .

Los mixins ```RoleMixin``` y ```UserMixin``` ya contienen atributos y métodos capaces de crear tablas sin necesidad de definirlas explícitamente y ligar los objetos instanciados a dichas tablas. Por ello se crearán las tablas ```rol``` y ```usuario``` sin necesidad de especificarlo.

El atributo ```User.roles``` utiliza la función ```db.relationship()``` para definir una relacion ente la clase ```User``` y la clase ```Role``` que se refleja en el objeto ```roles_users```. Es decir, un usuario puede tener múltiples roles y esta relación de roles es guardada en la tabla ```roles```, ligada al objeto ```roles_users```.

In [ ]:
class Role(db.Model, RoleMixin):
    __tablename__ = "role"
    id = db.Column(db.Integer(), primary_key=True)
    name = db.Column(db.String(80), unique=True)
    description = db.Column(db.String(255))

In [ ]:
class User(db.Model, UserMixin):
    __tablename__ = "user"
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(255), unique=True)
    nick = db.Column(db.String(25), unique=True)
    password = db.Column(db.String(255))
    active = db.Column(db.Boolean())
    roles = db.relationship('Role', secondary='roles_users',
                            backref=db.backref('users', lazy='dynamic'))

### Creación de las tablas en la base de datos.

Se crean las tablas en la base de datos.

In [ ]:
db.create_all()

Se puede comprobar el contenido de la base de datos de la siguiente manera:

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///data/alumnos.sqlite

### Creación del objeto que gestiona las operaciones de usuarios y roles.

La extensión *Flask-Security* permite crear un objeto capaz de utilizar las tablas recién creadas para las operaciones de gestión de usuarios y roles mediante la clase ```flask_security.SQLAlchemyUserDatastore```.

In [ ]:
datastore_user = SQLAlchemyUserDatastore(db, User, Role)

### Implementación de las operaciones de control de acceso  de la aplicación.

La clase ```Security``` permite crear un objeto el cual realiza las operaciones de control de acceso de la aplicación.

In [ ]:
seguridad = Security(app, datastore_user)

### Creación de un usuario nuevo.

In [ ]:
datastore_user.create_user(email="example@example.com", nick='admin', password='admin')
db.session.commit()

In [ ]:
%sql select * from user

### Creación de un rol nuevo.

In [ ]:
datastore_user.create_role(name='admin', description="Administrators")
db.session.commit()

In [ ]:
%sql select * from role

### Asignación de un rol.

In [ ]:
usuario = User.query.filter_by(id=1).first()

In [ ]:
usuario.nick

In [ ]:
usuario.password

In [ ]:
usuario.is_authenticated

In [ ]:
User.query.filter_by(nick="admin").first()

In [ ]:
rol = Role.query.filter_by(name='admin').first()
   db.session.commit()

In [ ]:
with app.test_client() as client:
    with app.app_context():
        print(usuario.get_auth_token())
        print(usuario.get_security_payload())

In [ ]:
%sql select * from roles_users

### Función de vista que requiere control de acceso mediante usuario y contraseña.

La función ```flask_security.login_required()``` aplicada como un decorador sobre la función ```home()``` indica que es necesario registrarse para pdoer acceder.

In [ ]:
@app.route('/')
@login_required
def home():
    print(dir(session))
    return session

### Ejecución de la aplicación.

Se ejecutará un servidor en http://localhost:5000 que pedirá un usuario y contraseña válidos para poder acceder.

**Advertencia:** Una vez ejecutada la siguente celda es necesario interrumpir el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run('0.0.0.0')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>